In [14]:
from pycocotools.coco import COCO

In [15]:
coco = COCO('gt1_coco_annotations.json')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [13]:
coco.loadAnns([0,1,2,3,4])

NameError: name 'coco' is not defined

In [16]:
import json 
with open('gt1_coco_annotations.json') as f:
    gt_coco = json.load(f)

In [17]:
for idx, ann in enumerate(gt_coco['annotations']):
    # print(ann)
    ann['id'] = idx

In [18]:
with open("gt1_coco_annotations.json","w") as f:
    json.dump(gt_coco,f,indent=4)

In [15]:
a= zip([],[])
a

In [5]:
import pickle

In [6]:
with open("/bd_byta6000i0/users/sam2/kyyang/SurgicalSAM2/wl_test/test/output/mask/prompt.pkl", "rb") as f:
    prompt_info = pickle.load(f)

In [7]:
len(prompt_info)

3

In [9]:
prompt_info[0]

{'prompt_objs': [{'mask': array([[ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          ...,
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False]]),
   'bbox': [0.0, 0.0, 129.0, 479.0],
   'points': [array([ 41, 447]), array([15, 60]), [805, 372], [284, 182]],
   'obj_id': 0,
   'pos_or_neg_label': [1, 1, 0, 0]},
  {'mask': array([[False, False, False, ...,  True,  True,  True],
          [False, False, False, ...,  True,  True,  True],
          [False, False, False, ...,  True,  True,  True],
          ...,
          [False, False, False, ...,  True,  True,  True],
          [False, False, False, ...,  True,  True,  True],
          [False, False, False, ...,  True,  True,  True]]),
   'bbox': [685.0, 0.0, 853.0, 479.0],
   'points': [array([8

In [10]:
with open("prompt.pkl", "rb") as f:
    prompt_1 = pickle.load(f)

In [11]:
prompt_1

[{'prompt_objs': [{'mask': array([[ True,  True,  True, ..., False, False, False],
           [ True,  True,  True, ..., False, False, False],
           [ True,  True,  True, ..., False, False, False],
           ...,
           [ True,  True,  True, ..., False, False, False],
           [ True,  True,  True, ..., False, False, False],
           [ True,  True,  True, ..., False, False, False]]),
    'bbox': [0.0, 0.0, 129.0, 479.0],
    'points': [array([28, 56]), array([ 41, 430]), [833, 99], [251, 135]],
    'obj_id': 0,
    'pos_or_neg_label': [1, 1, 0, 0]},
   {'mask': array([[False, False, False, ...,  True,  True,  True],
           [False, False, False, ...,  True,  True,  True],
           [False, False, False, ...,  True,  True,  True],
           ...,
           [False, False, False, ...,  True,  True,  True],
           [False, False, False, ...,  True,  True,  True],
           [False, False, False, ...,  True,  True,  True]]),
    'bbox': [685.0, 0.0, 853.0, 479.0],
    